In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('seaborn-bright')

import pickle
from sklearn.linear_model import Ridge, Lasso
from IPython.display import display
%load_ext memory_profiler
%load_ext line_profiler

In [ ]:
# not in notes (!)
df = pd.DataFrame(np.ones(50).reshape(10,5), 
                      index=pd.date_range('1/1/2016', periods=10, freq='H' ),
                      columns=list('ABCDE'))
df['size']=0
df['size'] = pd.Series([0,0,0,0,200,0,0,2000,0,0], index=df.index)
df.loc[:,"size_csum"] = df['size'].cumsum()

start = pd.to_datetime('1/1/1900')
df.loc[:,"size_astime"] = start + pd.to_timedelta(df.size_csum,'ms')
df.index = df.size_astime

display(df)
print(df.E.resample('1000ms',label='right').sum())

# timestamp_col = df.timestamp.resample('30min',label='right').last()

# pd.to_datetime(timestamp_col).dropna()

# after going around the circle, the workaround has to be applying an extra operation:
# `pd.to_datetime(timestamp_col).dropna()`

# df

In [ ]:
# flattening history for autoregression
df = pd.DataFrame({'val': [10,40,30,10,11,12,10,11,12,19]}, index=pd.date_range('2016-01-01 19:00:00', periods=10, freq='H' ) )
display(df)
def myf(df):
    for i in range(1,2):
        df['val_t-' + str(i)] = df['val'].shift(i)

    for i in range(1,5):
        df['d_val_t+' + str(i)] = df['val'].shift(-i) - df['val']

    df[['d_val_t+3','d_val_t+4']].mean(axis=1)
    return df

df.groupby(pd.TimeGrouper('D')).apply(myf)

In [23]:
%pdb off
def myf(lm, **args):
    args['cv'] = 'hep'
    print args
    model = lm(**args)
    X_train = [[1,2,2],[3,4,4]]
    y_train = [100,200]
    
    result = model.fit(X_train, y_train)  
    return result

my_model = Ridge
res = myf(my_model, alpha=0.1)
print res.coef_

my_model = Lasso
res = myf(my_model)
print res.coef_

Automatic pdb calling has been turned OFF
{'alpha': 0.1, 'cv': 'hep'}


TypeError: __init__() got an unexpected keyword argument 'cv'